In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import ujson as json
from collect_score_info import get_results
from utils import percent_data, \
    data_behaviour, \
    check_miss_results, \
    moe2name
import torch


from tqdm import tqdm
import numpy as np
from safetensors import safe_open
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator
# Set font globally to Times New Roman
plt.rcParams['font.family'] = 'DejaVu Serif'
fontsize = 14


In [2]:
import os
# variable constant
percent_data = [
#  'checkpoint-832',
 'checkpoint-3328',
#  'checkpoint-2496',
 'checkpoint-6656',
#  'checkpoint-4160',
 'checkpoint-9984',
 
#  'checkpoint-5824',
 
 'checkpoint-13312',
#  'checkpoint-7488',
 'checkpoint-16632'
]

# select benchmarks for select experts between layer on a amount of data
data_behaviour = [
    "mmbench_en_dev" ,
    'scienceqa_img', 
    'mmstar'
]


percentages = ['20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%']

data2name = {
    'scienceqa_img': 'scienceqa_img', 
    'mmstar': 'mmstar', 
    'mmbench_en_dev':'mmbench_en_dev'
}
'smoe_cosinegating', 'smoe', 'smoe_sigmoidgating', 'hyperrouter', 'smoe_perturbed'
moe2name = {
    "smoe_cosinegating": "Cosine Router",
    "smoe_sigmoidgating" : "Sigmoid Router",
    "hyperrouter": "Hyper Router",
    "smoe_perturbed": "Perturbed Cosine Router",
    "smoe": "SMoE Router"

}

def check_miss_results(results):
    # pass
    # check weght had miss evaluate 
    for stage in results['phi3mini-clip'].keys():
        for data_name in data_behaviour:

            for percent in percent_data:
                try:
                    x =  results['phi3mini-clip'][stage][data_name][percent]
                except:
                    pass
                    # print(f"stage: {stage} - data name: {data_name} - percent {percent}")

In [10]:
from utils import check_exist_id_experts
import glob
import json
from tqdm import tqdm  # Import tqdm


def get_results(path_logs_results_list, path_save="./results/result_metric.json"):
    logs = []
    
    # Gather logs from multiple paths using a loop
    for path_logs_results in path_logs_results_list:
        logs.extend(glob.glob(path_logs_results))

    results = {}

    # Use tqdm to show progress on logs
    for path_logs in tqdm(logs, desc="Processing logs"):
        path_insight = f"{path_logs}/results.json"
        path_experts = f"{path_logs}/"
        try:
            with open(path_insight, "r") as f:
                data_logs = json.load(f)
        except:
            continue

        model = data_logs['model_configs']['model_args']

        # check exist metrict 10%
        check = None

        for name_percent in percent_data:
            if name_percent in model:
                check = name_percent
        if check == None or "dense" in model or "Xphi35-siglip224" not in model:
            continue
        if "dynamic" in path_insight:
            print(path_insight)
        name_model = model.split("/")[-2]
        stage = model.split("/")[-1]
        if name_model not in results:
            results[name_model] = {}
        for data_name in list(data_logs['results'].keys()):
            if stage not in results:
                results[stage] = {}
            if data_name not in results[stage]:
                results[stage][data_name] = []
         
            results[stage][data_name].append(check)
        
    return results



path_logs_results = [
    # "/cm/archive/anonymous/toolkitmoe/evaluate/logs/*",
    "/cm/shared/anonymous_H102/toolkitmoe/evaluate/logs/*",
    # "/cm/archive/anonymous/toolkitmoe/evaluate/logs/*",
    # "/cm/archive/anonymous/toolkitmoe/evaluate/logs/logs/*",
]

results = get_results(path_logs_results)


Processing logs: 100%|██████████| 1194/1194 [00:01<00:00, 831.51it/s]


In [11]:
results

{'Full_new_smoe_sigmoidgating': {},
 'checkpoint-3328,conv_template=phi35': {'gqa': ['checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328'],
  'mmerealworld_lite': ['checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328'],
  'mmbench_en_dev': ['checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328'],
  'ai2d': ['checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328',
   'checkpoint-3328'],
  'mmmu_val': ['checkpoint-3328',
   'checkpoint-3328

In [4]:
data_need_evals = [
    "mmbench_en_dev", 
    "mmstar", 
    "ai2d", 
    "mmmu_val", 
    "textvqa_val",
    "gqa", 
    "ocrbench", 
    "pope", 
    # "mathvista_testmini", 
    # "hallusion_bench_image",
    "mmerealworld_lite"
]
#"ai2d,mmbench_en_dev,mmmu_val,mmstar,ocrbench,textvqa_val,mmerealworld_lite,pope"
moe_methods = ['Full_competesmoev30', 'Full_xmoe', 'Full_new_smoe_sigmoidgating', 'Full_new_smoe_share', "Full_smoe_perturbed", "Full_smoe"]

In [5]:
for data in data_need_evals:
    path_origin = "phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating"
    for name_percent in percent_data:
        print(f"'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/{name_percent}|{data}'")


'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-3328|mmbench_en_dev'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-6656|mmbench_en_dev'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-9984|mmbench_en_dev'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-13312|mmbench_en_dev'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-16632|mmbench_en_dev'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-3328|mmstar'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-6656|mmstar'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-9984|mmstar'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/smoe_sigmoidgating/checkpoint-13312|mmstar'
'phi3|/cm/archive/anonymous/checkpoints/phi3mini

In [7]:
miss_data = []
for moe_method in moe_methods:
    
    for data in data_need_evals:
        if data not in results[moe_method]:
            results[moe_method][data] = []
        for name_percent in percent_data:
            try:
                if name_percent not in results[moe_method][data]:
                    miss_data.append((moe_method, data, name_percent))
            except:
                print(moe_method, data)


KeyError: 'Full_competesmoev30'

In [ ]:
miss_data

[('hyperrouter', 'ai2d', 'checkpoint-832')]

In [ ]:
path_origin = "phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft"
miss_details = {}
nb_miss = []
for info in miss_data:
    if info[0] not in miss_details: miss_details[info[0] ] = []
    path = f"{path_origin}/{info[0]}/{info[2]}|{info[1]}"
    miss_details[info[0] ].append(path)
    nb_miss.append(path)

In [ ]:
miss_data

[('hyperrouter', 'ai2d', 'checkpoint-832')]

In [ ]:
len(np.unique(nb_miss))

1

In [ ]:
miss_details

{'hyperrouter': ['phi3|/cm/archive/anonymous/checkpoints/phi3mini-clip/sft/hyperrouter/checkpoint-832|ai2d']}